# Transformation Bronze → Silver

##  Objectif
Nettoyer, standardiser et enrichir les données brutes du Lakehouse Bronze.

##  Transformations appliquées
1. **Nettoyage numérique** : Arrondi à 2 décimales
2. **Enrichissement temporel** : Extraction jour/mois/année/trimestre
3. **Standardisation** : Correction du format de time
4. **Calcul métier** : Période de la journée basée sur l'heure

##  Dépendances
- **Input** : `LH_Wind_Power_Bronze.dbo.wind_power`
- **Output** : `LH_Wind_Power_Silver.dbo.wind_power`

##  Mode de sauvegarde
- **Mode** : Overwrite (écrasement complet)
- **Raison** : Simplicité pour ce projet pédagogique

# 2.2 - Cellule 1 : Imports

In [6]:
from pyspark.sql.functions import (
    round, col, dayofmonth, month, year, quarter, 
    substring, when, regexp_replace
)

StatementMeta(, 2394bd20-a66f-4d45-877d-3cce4bc0b230, 8, Finished, Available, Finished)

# 2.3 - Cellule 2 : Charger les données depuis Bronze

In [4]:
# Chemin vers la table Bronze
bronze_table_path = "abfss://44fadaf6-616a-4041-8a61-e92d278f3039@onelake.dfs.fabric.microsoft.com/3b5e9453-5d0e-43b9-800f-132c10130c48/Tables/dbo/wind_power"# Chemin vers la table Bronze

# Charger les données
df = spark.read.format("delta").load(bronze_table_path)

# Afficher le schéma et un aperçu
print(" Schéma des données Bronze :")
df.printSchema()

print(f"\n Nombre de lignes : {df.count()}")

print("\n Aperçu des 5 premières lignes :")
df.show(5, truncate=False)

StatementMeta(, 2394bd20-a66f-4d45-877d-3cce4bc0b230, 6, Finished, Available, Finished)

 Schéma des données Bronze :
root
 |-- production_id: long (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- time: string (nullable = true)
 |-- turbine_name: string (nullable = true)
 |-- capacity: long (nullable = true)
 |-- location_name: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- region: string (nullable = true)
 |-- status: string (nullable = true)
 |-- responsible_department: string (nullable = true)
 |-- wind_speed: double (nullable = true)
 |-- wind_direction: string (nullable = true)
 |-- energy_produced: double (nullable = true)


 Nombre de lignes : 1296

 Aperçu des 5 premières lignes :
+-------------+-------------------+--------+------------+--------+-------------+--------+---------+--------+----------------------+----------------------+----------+--------------+---------------+
|production_id|date               |time    |turbine_name|capacity|location_name|latitude|longitude|region  |status   

# 2.4 - Cellule 3 : Appliquer les transformations

In [7]:
# Appliquer toutes les transformations en une seule opération chaînée
df_transformed = (df
    # Arrondir les valeurs numériques à 2 décimales
    .withColumn("wind_speed", round(col("wind_speed"), 2))
    .withColumn("energy_produced", round(col("energy_produced"), 2))
    
    #  Extraire les composants de date
    .withColumn("day", dayofmonth(col("date")))
    .withColumn("month", month(col("date")))
    .withColumn("quarter", quarter(col("date")))
    .withColumn("year", year(col("date")))
    
    # 🕐Corriger le format de time (remplacer - par :)
    .withColumn("time", regexp_replace(col("time"), "-", ":"))
    
    #  Extraire les composants de temps
    .withColumn("hour_of_day", substring(col("time"), 1, 2).cast("int"))
    .withColumn("minute_of_hour", substring(col("time"), 4, 2).cast("int"))
    .withColumn("second_of_minute", substring(col("time"), 7, 2).cast("int"))
    
    #  Calculer la période de la journée
    .withColumn("time_period", 
        when((col("hour_of_day") >= 5) & (col("hour_of_day") < 12), "Morning")
        .when((col("hour_of_day") >= 12) & (col("hour_of_day") < 17), "Afternoon")
        .when((col("hour_of_day") >= 17) & (col("hour_of_day") < 21), "Evening")
        .otherwise("Night")
    )
)

print(" Transformations appliquées avec succès !")

StatementMeta(, 2394bd20-a66f-4d45-877d-3cce4bc0b230, 9, Finished, Available, Finished)

 Transformations appliquées avec succès !


# 2.5 - Cellule 4 : Afficher un échantillon transformé

In [4]:
# Afficher un échantillon des données transformées
print(" Aperçu des données transformées :")
df_transformed.select(
    "date", "time", "turbine_name", 
    "wind_speed", "energy_produced", 
    "day", "month", "year", "quarter",
    "hour_of_day", "time_period"
).show(10)

print(f"\n Nombre de colonnes : {len(df_transformed.columns)}")
print(f" Nouvelles colonnes ajoutées : day, month, quarter, year, hour_of_day, minute_of_hour, second_of_minute, time_period")

StatementMeta(, d2565976-4709-434e-8237-73c44e4746b6, 6, Finished, Available, Finished)

 Aperçu des données transformées :
+-------------------+--------+------------+----------+---------------+---+-----+----+-------+-----------+-----------+
|               date|    time|turbine_name|wind_speed|energy_produced|day|month|year|quarter|hour_of_day|time_period|
+-------------------+--------+------------+----------+---------------+---+-----+----+-------+-----------+-----------+
|2024-06-16 00:00:00|09:00:00|   Turbine A|     10.14|         967.44| 16|    6|2024|      2|          9|    Morning|
|2024-06-16 00:00:00|09:00:00|   Turbine B|      9.52|        1887.41| 16|    6|2024|      2|          9|    Morning|
|2024-06-16 00:00:00|09:00:00|   Turbine C|      5.85|         979.52| 16|    6|2024|      2|          9|    Morning|
|2024-06-16 00:00:00|09:10:00|   Turbine A|     24.54|         905.83| 16|    6|2024|      2|          9|    Morning|
|2024-06-16 00:00:00|09:10:00|   Turbine B|     13.86|         642.42| 16|    6|2024|      2|          9|    Morning|
|2024-06-16 00:00:00|

# 2.6 - Cellule 5 : Vérifications de qualité

In [5]:
from pyspark.sql.functions import count, when, isnan, col, min as spark_min, max as spark_max

# Vérifier qu'il n'y a pas de valeurs nulles dans les colonnes critiques
print("===  Vérification des valeurs nulles ===")
null_counts = df_transformed.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in ["wind_speed", "energy_produced", "day", "month", "year", "time_period"]
])
null_counts.show()

# Vérifier les valeurs uniques de time_period
print("\n===  Distribution des périodes de la journée ===")
df_transformed.groupBy("time_period").count().orderBy("count", ascending=False).show()

# Vérifier les plages de dates
print("\n===  Plage de dates ===")
df_transformed.select(
    spark_min("date").alias("Date minimale"),
    spark_max("date").alias("Date maximale")
).show()

# Statistiques descriptives
print("\n===  Statistiques sur les mesures ===")
df_transformed.select("wind_speed", "energy_produced").describe().show()

StatementMeta(, d2565976-4709-434e-8237-73c44e4746b6, 7, Finished, Available, Finished)

===  Vérification des valeurs nulles ===
+----------+---------------+---+-----+----+-----------+
|wind_speed|energy_produced|day|month|year|time_period|
+----------+---------------+---+-----+----+-----------+
|         0|              0|  0|    0|   0|          0|
+----------+---------------+---+-----+----+-----------+


===  Distribution des périodes de la journée ===
+-----------+-----+
|time_period|count|
+-----------+-----+
|      Night|  432|
|    Morning|  378|
|  Afternoon|  270|
|    Evening|  216|
+-----------+-----+


===  Plage de dates ===
+-------------------+-------------------+
|      Date minimale|      Date maximale|
+-------------------+-------------------+
|2024-06-15 00:00:00|2024-06-16 00:00:00|
+-------------------+-------------------+


===  Statistiques sur les mesures ===
+-------+------------------+------------------+
|summary|        wind_speed|   energy_produced|
+-------+------------------+------------------+
|  count|              1296|              1296|


# 2.7 - Cellule 6 : Sauvegarder dans Silver

In [8]:
# Chemin vers la table Silver
silver_table_path = "abfss://44fadaf6-616a-4041-8a61-e92d278f3039@onelake.dfs.fabric.microsoft.com/0af8fbde-081f-47d6-9945-e8ea30aa73ab/Tables/dbo/wind_power"


# Sauvegarder en mode overwrite (écrasement complet)
df_transformed.write.format("delta").mode("overwrite").save(silver_table_path)

print(" Données transformées et sauvegardées dans Silver")
print(f" Nombre de lignes sauvegardées : {df_transformed.count()}")

StatementMeta(, 2394bd20-a66f-4d45-877d-3cce4bc0b230, 10, Finished, Available, Finished)

✅ Données transformées et sauvegardées dans Silver
📊 Nombre de lignes sauvegardées : 1296
